# Loop strength calculation

## Imports

In [ ]:
# import standard python libraries
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 96
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# import libraries for biological data analysis
from coolpuppy import coolpup
from plotpuppy import plotpup
import cooler
import bioframe
import cooltools
from cooltools import expected_cis
from cooltools.lib import plotting

import bbi

## Inputs

First, get the loops and mcools to analyse, set up variables, etc

In [ ]:
#mcool resolution to read
resolution = 250
#List of mcool locations as strings
conditions = ["mcoollocation1", "mcoollocation2", "mcoollocation3"]
#List of loop types as strings
loopTypesNames = ["loop", "type", "names"]
#List of loop file locations (bedpe)
loopFiles = ["looplocation1", "looplocation2", "looplocation3"]

#Specify the RCMC regions of the mcools to look at (format: chromosome (string), start (number), end (number), name of region (string))
regions = pd.DataFrame([['chrA',1,100,'regionname1'],['chrB',1,100,'regionname2'],['chrC',1,100,'regionname3']],
                  columns=['chrom', 'start', 'end', 'name'])
#Cis expected file locations from cooltools - .tsv file - one for each mcool
expectedFiles = ["expectedlocation1", "expectedlocation2", "expectedlocation3"]
#Set save directory
saveDir = '/a/directory/on/your/system/'

#Set the size of the area flanking the dot
flankDist = 10000
#Don't set this to be even... This is the size of the area to measure around the dot 
#(and by extension the size of the boxes at the edges of the region too)
#For this reason, it needs to be odd to have integer box sizes on each side.
dotWindow = 5


Run the imports

In [ ]:
#######Don't change this section#######
#Creat an empty list to store the imported loop locations
loopTypes = []
#List of column names to use for imported loops (this is constant - do not change)
colNames = ['chrom1', 'start1', 'end1', 'chrom2', 'start2', 'end2']
#Read in files, put them in loopTypes
for file in loopFiles:
    temploops = pd.read_csv(file, sep='\t', names=colNames, header=None)
    loopTypes.append(temploops)

## Enrichment calculation

In [ ]:
## Enrichment calculation

#Viraat's new calculation
#Modified 2022/10/04 by Miles to try to avoid NaN values and correct an issue with the background sum, 
#and generally make the code a little more streamlined
def enrichmentCalc(mtx, dotWindow):
    #Dimension of array side (should be square)
    sideLength = len(mtx)
    #Middle of side length
    midPoint = (sideLength - 1) // 2
    #Half size of box around centre pixel (one pixel smaller if even-sized dot window - don't do this)
    buffer = (dotWindow - 1) // 2
    
    #Get sum of pixels around dot
    dotSum = np.nansum(mtx[midPoint-buffer:midPoint+buffer+1, midPoint-buffer:midPoint+buffer+1])
    
    #Subset the matrix and calculate the mean without NaN values
    backgroundSum1 = np.nansum(mtx[0:dotWindow, 0:dotWindow])
    backgroundSum2 = np.nansum(mtx[sideLength-dotWindow:sideLength, sideLength-dotWindow:sideLength])
    
    #Calculate enrichment (NB this assumes all boxes are the same size.
    #If you set an even dotWindow value, they won't be)
    enrichment = dotSum / ((backgroundSum1 + backgroundSum2)/2)
    
    return enrichment

# Get the strengths

Function for getting strength of each loop (uses the pileup function from cooltools to do observed/expected)

In [ ]:
def loopStrengthGet(loop, flankDist, clr, regions, expected, dotWindow):
    loopf = loop.to_frame().T
    loopf = loopf.astype({'start1':'int64','end1':'int64','start2':'int64','end2':'int64'})
    stack = cooltools.pileup(clr, loopf, view_df=regions, expected_df=expected, flank=flankDist)
    mtx = np.nanmean(stack, axis=2)
    enrichment = enrichmentCalc(mtx, dotWindow)
    
    return enrichment

In [ ]:
#Zip the names and loop info into a dictionary for easier referencing
loopDict = dict(zip(loopTypesNames, loopTypes))
#Stop the code if you used an even value for dotWindow, since it won't work
if dotWindow % 2 == 0:
    print("You need to use an odd number for dotWindow in the inputs section")
else:
    #Loop through the conditions
    for i, condition in enumerate(conditions):
        #Get the cooler data
        clr = cooler.Cooler(condition+'::/resolutions/'+str(resolution))
        #Get the corresponding expected data
        expected = pd.read_csv(expectedFiles[i], sep='\t')

        #Loop through loopDict
        for loopsName in loopDict:
            #Read out the loops
            loops = loopDict[loopsName]
            #For each row (ie loop), do pileup, get enrichment, write to new column [condition]_strength
            loops[f'{condition}_strength'] = loops.apply(loopStrengthGet, axis = 1, flankDist = flankDist, clr = clr, regions = regions, expected = expected, dotWindow = dotWindow)

            loopDict[loopsName] = loops
        

## Output files - one for each loop type

In [ ]:
for name, df in loopDict.items():
    df.to_csv(saveDir + name + '.bedpe', sep = '\t', index = False, header = True)